![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)

# <h1><center>チュートリアル  <a href="https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

このノートブックは、ネイティブPythonバインディングを使用した [**MuJoCo** physics](https://github.com/google-deepmind/mujoco#readme) の入門チュートリアルを提供します。

<!-- Copyright 2021 DeepMind Technologies Limited

     Licensed under the Apache License, Version 2.0 (the "License");
     you may not use this file except in compliance with the License.
     You may obtain a copy of the License at

         http://www.apache.org/licenses/LICENSE-2.0

     Unless required by applicable law or agreed to in writing, software
     distributed under the License is distributed on an "AS IS" BASIS,
     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
     See the License for the specific language governing permissions and
     limitations under the License.
-->

# すべてのインポート

In [0]:
!pip install mujoco

# GPUレンダリングをセットアップ
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# glvndがNvidia EGLドライバを検出できるようにICDコンフィグを追加します。
# これは通常Nvidiaドライバパッケージの一部としてインストールされますが、Colabの
# カーネルはAPT経由でドライバをインストールしないため、ICDが欠けています。
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# MuJoCoがEGLレンダリングバックエンドを使用するように設定（GPU必須）
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# インストールが成功したか確認
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# その他のインポートとヘルパー関数
import time
import itertools
import numpy as np

# グラフィックとプロット
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# numpyの出力をより読みやすく
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()


# MuJoCo の基礎

まず、シンプルなモデルを定義して読み込みます。

In [0]:
xml = """
<mujoco>
  <worldbody>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)

`xml` 文字列は、MuJoCoの [MJCF](http://www.mujoco.org/book/modeling.html) で記述されています。これは [XML](https://en.wikipedia.org/wiki/XML#Key_terminology) ベースのモデリング言語です。
  - 必須要素は `<mujoco>` のみです。最小の有効なMJCFモデルは、完全に空のモデルである `<mujoco/>` です。
  - すべての物理要素は `<worldbody>` の内部に存在します。これは常にトップレベルのボディであり、デカルト座標でのグローバル原点を構成します。
  - worldに `red_box` と `green_sphere` という名前の2つのgeomを定義します。
  - **質問:** `red_box` には位置がなく、`green_sphere` にはタイプがありません。なぜでしょうか？
    - **答え:** MJCF属性には*デフォルト値*があります。デフォルトの位置は `0 0 0` で、デフォルトのgeomタイプは `sphere` です。MJCF言語は、ドキュメントの [XML Reference章](https://mujoco.readthedocs.io/en/latest/XMLreference.html) に記載されています。

`from_xml_string()` メソッドはモデルコンパイラを呼び出し、バイナリの `mjModel` インスタンスを作成します。

## mjModel

MuJoCoの `mjModel` には、*モデル記述*、つまり*時間とともに変化しない*すべての量が含まれています。`mjModel` の完全な説明は、ヘッダーファイル [`mjmodel.h`](https://github.com/google-deepmind/mujoco/blob/main/include/mujoco/mjmodel.h) の末尾にあります。ヘッダーファイルには、各フィールドを説明する短くて有用なインラインコメントが含まれていることに注意してください。

`mjModel` で見つけることができる量の例としては、シーン内のgeomの数である `ngeom` や、それぞれの色である `geom_rgba` があります。

In [0]:
model.ngeom

In [0]:
model.geom_rgba

## 名前によるアクセス

MuJoCoのPythonバインディングは、名前を使用した便利な [アクセッサ](https://mujoco.readthedocs.io/en/latest/python.html#named-access) を提供します。名前文字列なしで `model.geom()` アクセッサを呼び出すと、有効な名前が何であるかを教えてくれる便利なエラーが生成されます。

In [0]:
try:
  model.geom()
except KeyError as e:
  print(e)

プロパティを指定せずに名前付きアクセッサを呼び出すと、すべての有効なプロパティが何であるかを教えてくれます。

In [0]:
model.geom('green_sphere')

`green_sphere` のrgba値を読み取ってみましょう。

In [0]:
model.geom('green_sphere').rgba

この機能は、MuJoCoの [`mj_name2id`](https://mujoco.readthedocs.io/en/latest/APIreference.html?highlight=mj_name2id#mj-name2id) 関数の便利なショートカットです。

In [0]:
id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, 'green_sphere')
model.geom_rgba[id, :]

同様に、読み取り専用の `id` および `name` プロパティを使用して、idから名前に、またその逆に変換できます。

In [0]:
print('id of "green_sphere": ', model.geom('green_sphere').id)
print('name of geom 1: ', model.geom(1).name)
print('name of body 0: ', model.body(0).name)

0番目のボディは常に `world` であることに注意してください。これは名前を変更できません。

`id` および `name` 属性は、Python内包表記で便利です。

In [0]:
[model.geom(i).name for i in range(model.ngeom)]

## `mjData`
`mjData` には、*状態*とそれに依存する量が含まれます。状態は、時間、[一般化](https://en.wikipedia.org/wiki/Generalized_coordinates) 位置、一般化速度で構成されます。これらはそれぞれ `data.time`、`data.qpos`、`data.qvel` です。新しい `mjData` を作成するには、`mjModel` があればよいです。

In [0]:
data = mujoco.MjData(model)

`mjData` には *状態の関数* も含まれます。たとえば、ワールドフレーム内のオブジェクトのデカルト位置です。2つのgeomの (x, y, z) 位置は `data.geom_xpos` にあります。

In [0]:
print(data.geom_xpos)

待ってください、なぜ両方のgeomが原点にあるのでしょうか？緑の球をオフセットしませんでしたか？答えは、`mjData` の派生量は明示的に伝播する必要があるということです（[下記](#scrollTo=QY1gpms1HXeN) 参照）。私たちの場合、最小限必要な関数は [`mj_kinematics`](https://mujoco.readthedocs.io/en/latest/APIreference.html#mj-kinematics) で、これはすべてのオブジェクト（カメラとライトを除く）のグローバルデカルトポーズを計算します。

In [0]:
mujoco.mj_kinematics(model, data)
print('raw access:\n', data.geom_xpos)

# MjDataも名前付きアクセスをサポートしています
print('\nnamed access:\n', data.geom('green_sphere').xpos)

# 基本的なレンダリング、シミュレーション、アニメーション

レンダリングするには、`Renderer` オブジェクトをインスタンス化して、その `render` メソッドを呼び出す必要があります。

また、colabのセクションを独立させるためにモデルを再読み込みします。

In [0]:
xml = """
<mujoco>
  <worldbody>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
# モデルとデータを作成
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# レンダラーを作成し、レンダリングしてピクセルを表示
with mujoco.Renderer(model) as renderer:
  media.show_image(renderer.render())

うーん、なぜ黒いピクセルなのでしょうか？

**答え:** 上記と同じ理由で、まず `mjData` の値を伝播する必要があります。今回は [`mj_forward`](https://mujoco.readthedocs.io/en/latest/APIreference/APIfunctions.html#mj-forward) を呼び出します。これは加速度の計算までパイプライン全体を呼び出します。つまり、$\dot x = f(x)$ を計算します。ここで $x$ は状態です。この関数は実際に必要なものよりも多くのことを行いますが、計算時間を節約することを気にしない限り、`mj_forward` を呼び出すのが良い習慣です。そうすれば、何も見逃していないことがわかります。

また、レンダラーが保持する視覚シーンを記述するオブジェクトである `mjvScene` を更新する必要があります。後で、シーンには物理モデルの一部ではない視覚オブジェクトを含めることができることを見ます。

In [0]:
with mujoco.Renderer(model) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data)

  media.show_image(renderer.render())

これでうまくいきましたが、この画像は少し暗いです。ライトを追加して再レンダリングしましょう。

In [0]:
xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

with mujoco.Renderer(model) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data)

  media.show_image(renderer.render())

ずっと良くなりました！

`mjModel` インスタンスのすべての値は書き込み可能であることに注意してください。一般的にはこれを行わず、XMLで値を変更することをお勧めしますが（無効なモデルを作りやすいため）、一部の値は安全に書き込めます。たとえば、色などです。

In [0]:
# 異なる色を表示するには、このセルを複数回実行してください
model.geom('red_box').rgba[:3] = np.random.rand(3)
with mujoco.Renderer(model) as renderer:
  renderer.update_scene(data)

  media.show_image(renderer.render())

# シミュレーション

それでは、シミュレーションを実行してビデオを作成しましょう。MuJoCoのメイン高レベル関数である `mj_step` を使用します。これは状態を $x_{t+h} = f(x_t)$ とステップさせます。

下のコードブロックでは、各 `mj_step` 呼び出しの後にレンダリングしていない*ことに注意してください。これは、デフォルトのタイムステップが2msであり、500fpsではなく60fpsのビデオが必要だからです。

In [0]:
duration = 3.8  # (秒)
framerate = 60  # (Hz)

# シミュレートしてビデオを表示
frames = []
mujoco.mj_resetData(model, data)  # 状態と時間をリセット
with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

うーん、ビデオは再生されていますが、何も動いていません。なぜでしょうか？

これは、このモデルに [自由度](https://www.google.com/url?sa=D&q=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FDegrees_of_freedom_(mechanics))（DoFs）がないためです。動くもの（そして慣性を持つもの）は*ボディ*と呼ばれます。ボディに*ジョイント*を追加して、親に対してどのように動くことができるかを指定することで、DoFsを追加します。新しいボディを作成してgeomを含め、ヒンジジョイントを追加して再レンダリングしましょう。可視化オプションオブジェクト `MjvOption` を使用してジョイント軸を可視化します。

In [0]:
xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <body name="box_and_sphere" euler="0 0 -30">
      <joint name="swing" type="hinge" axis="1 -1 0" pos="-.2 -.2 -.2"/>
      <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
      <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
    </body>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# ジョイント可視化オプションを有効化
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

duration = 3.8  # (秒)
framerate = 60  # (Hz)

# シミュレートしてビデオを表示
frames = []
mujoco.mj_resetData(model, data)
with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=scene_option)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

`box_and_sphere` ボディをZ（垂直）軸を中心に30°回転させたことに注意してください。これは `euler="0 0 -30"` ディレクティブで行いました。これは、[キネマティックツリー](https://www.google.com/url?sa=D&q=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FKinematic_chain) 内の要素のポーズは常に*親ボディ*に対して相対的であることを強調するために行われました。そのため、2つのgeomもこの変換によって回転しました。

物理オプションは `mjModel.opt` にあります。たとえば、タイムステップなどです。

In [0]:
model.opt.timestep

重力を反転して再レンダリングしましょう。

In [0]:
print('default gravity', model.opt.gravity)
model.opt.gravity = (0, 0, 10)
print('flipped gravity', model.opt.gravity)

# シミュレートしてビデオを表示
frames = []
mujoco.mj_resetData(model, data)
with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=scene_option)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=60)

これをXMLでトップレベルの `<option>` 要素を使用して行うこともできます。
```xml
<mujoco>
  <option gravity="0 0 10"/>
  ...
</mujoco>
```

### 自由度の理解

現実世界では、すべての剛体は6つの自由度を持っています。3つの並進と3つの回転です。現実世界のジョイントは制約として機能し、ジョイントで接続されたボディから相対的な自由度を取り除きます。一部の物理シミュレーションソフトウェアはこの表現を使用しており、これは「デカルト」または「減算」表現として知られていますが、これは非効率的です。MuJoCoは「ラグランジアン」、「一般化」、または「加算」表現として知られる表現を使用します。これにより、オブジェクトはジョイントを使用して明示的に追加されない限り自由度を持ちません。

1つのヒンジジョイントを持つ私たちのモデルには、1つの自由度があり、全状態はこのジョイントの角度と角速度によって定義されます。これらがシステムの一般化位置と速度です。

In [0]:
print('Total number of DoFs in the model:', model.nv)
print('Generalized positions:', data.qpos)
print('Generalized velocities:', data.qvel)

MuJoCoが一般化座標を使用しているため、レンダリングまたはオブジェクトのグローバルポーズを読み取る前に関数（例えば [`mj_forward`](https://mujoco.readthedocs.io/en/latest/APIreference.html#mj-forward)）を呼び出す必要があります。デカルト位置は一般化位置から*派生*し、明示的に計算する必要があります。

# 例：自己反転する「tippe-top」で自由ボディをシミュレートする

自由ボディは、6DoFsを持つ [フリージョイント](https://www.google.com/url?sa=D&q=https%3A%2F%2Fmujoco.readthedocs.io%2Fen%2Flatest%2FXMLreference.html%3Fhighlight%3Dfreejoint%23body-freejoint) を持つボディです。つまり、3つの並進と3つの回転です。`box_and_sphere` ボディにフリージョイントを与えて落下を見ることもできますが、もっと面白いものを見てみましょう。「tippe top」は、自分自身を反転させる回転おもちゃです（[ビデオ](https://www.youtube.com/watch?v=kbYpVrdcszQ)、[Wikipedia](https://en.wikipedia.org/wiki/Tippe_top)）。次のようにモデル化します。

In [0]:
tippe_top = """
<mujoco model="tippe top">
  <option integrator="RK4"/>

  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300"/>
    <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
  </asset>

  <worldbody>
    <geom size=".2 .2 .01" type="plane" material="grid"/>
    <light pos="0 0 .6"/>
    <camera name="closeup" pos="0 -.1 .07" xyaxes="1 0 0 0 1 2"/>
    <body name="top" pos="0 0 .02">
      <freejoint/>
      <geom name="ball" type="sphere" size=".02" />
      <geom name="stem" type="cylinder" pos="0 0 .02" size="0.004 .008"/>
      <geom name="ballast" type="box" size=".023 .023 0.005"  pos="0 0 -.015"
       contype="0" conaffinity="0" group="3"/>
    </body>
  </worldbody>

  <keyframe>
    <key name="spinning" qpos="0 0 0.02 1 0 0 0" qvel="0 0 0 0 1 200" />
  </keyframe>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(tippe_top)
data = mujoco.MjData(model)

mujoco.mj_forward(model, data)
with mujoco.Renderer(model) as renderer:
  renderer.update_scene(data, camera="closeup")

  media.show_image(renderer.render())

このモデル定義のいくつかの新機能に注意してください。
1. 6-DoFフリージョイントは `<freejoint/>` 句で追加されます。
2. `<option/>` 句を使用して、インテグレータを4次 [Runge Kutta](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) に設定します。Runge-Kuttaはデフォルトのオイラーインテグレータよりも収束率が高く、多くの場合、所定のタイムステップサイズでの精度が向上します。
3. `<asset/>` 句内でフロアのグリッドマテリアルを定義し、`"floor"` geomでそれを参照します。
4. トップの重心を下げるために、`ballast` という不可視で非衝突のボックスgeomを使用します。反転動作が発生するには、重心が低いことが（直感に反して）必要です。
5. 初期回転状態を*キーフレーム*として保存します。Z軸を中心とした高い回転速度を持っていますが、完全にワールドと向きが揃っていないため、反転に必要な対称性の破れが導入されます。
6. モデルで `<camera>` を定義し、`update_scene()` への `camera` 引数を使用してそこからレンダリングします。
状態を調べてみましょう。



In [0]:
print('positions', data.qpos)
print('velocities', data.qvel)

速度は解釈しやすく、各DoFに対して1つずつ、6つのゼロです。長さ7の位置は何でしょうか？ボディの初期2cmの高さを見ることができます。その後の4つの数値は、*単位四元数*によって定義される3D方向です。3D方向は**4**個の数値で表されますが、角速度は**3**個の数値です。詳細については、[四元数と空間回転](https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation) に関するWikipedia記事を参照してください。

ビデオを作りましょう。

In [0]:
duration = 7    # (秒)
framerate = 60  # (Hz)

# シミュレートしてビデオを表示
frames = []
mujoco.mj_resetDataKeyframe(model, data, 0)  # 状態をキーフレーム0にリセット
with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, "closeup")
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

### `mjData` からの値の測定
上記で述べたように、`mjData` 構造には、動的変数とシミュレーションによって生成される中間結果が含まれており、これらは各タイムステップで*変化することが予想されます*。以下では、2000タイムステップシミュレートし、トップの角速度と茎の高さを時間の関数としてプロットします。

In [0]:
timevals = []
angular_velocity = []
stem_height = []

# シミュレートしてデータを保存
mujoco.mj_resetDataKeyframe(model, data, 0)
while data.time < duration:
  mujoco.mj_step(model, data)
  timevals.append(data.time)
  angular_velocity.append(data.qvel[3:6].copy())
  stem_height.append(data.geom_xpos[2,2]);

dpi = 120
width = 600
height = 800
figsize = (width / dpi, height / dpi)
_, ax = plt.subplots(2, 1, figsize=figsize, dpi=dpi, sharex=True)

ax[0].plot(timevals, angular_velocity)
ax[0].set_title('angular velocity')
ax[0].set_ylabel('radians / second')

ax[1].plot(timevals, stem_height)
ax[1].set_xlabel('time (seconds)')
ax[1].set_ylabel('meters')
_ = ax[1].set_title('stem height')

# 例：カオス振り子

以下は、サンフランシスコのExploratoriumにある [これ](https://www.exploratorium.edu/exhibits/chaotic-pendulum) と似たカオス振り子のモデルです。

In [0]:
chaotic_pendulum = """
<mujoco>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>
    <body name="0" pos="0 0 .2">
      <joint name="root"/>
      <geom fromto="-.2 0 0 .2 0 0" rgba="1 1 0 1"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>
      <body name="1" pos="-.2 0 0">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="1 0 0 1"/>
      </body>
      <body name="2" pos=".2 0 0">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 1 0 1"/>
      </body>
      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(chaotic_pendulum)
data = mujoco.MjData(model)
height = 480
width = 640

with mujoco.Renderer(model, height, width) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data, camera="fixed")

  media.show_image(renderer.render())

## タイミング
動作中のビデオを見ながら、コンポーネントの時間を測定してみましょう。

In [0]:
# セットアップ
n_seconds = 6
framerate = 30  # Hz
n_frames = int(n_seconds * framerate)
frames = []
height = 240
width = 320

# 初期状態を設定
mujoco.mj_resetData(model, data)
data.joint('root').qvel = 10

# シミュレートしてフレームを記録
frame = 0
sim_time = 0
render_time = 0
n_steps = 0
with mujoco.Renderer(model, height, width) as renderer:
  for i in range(n_frames):
    while data.time * framerate < i:
      tic = time.time()
      mujoco.mj_step(model, data)
      sim_time += time.time() - tic
      n_steps += 1
    tic = time.time()
    renderer.update_scene(data, "fixed")
    frame = renderer.render()
    render_time += time.time() - tic
    frames.append(frame)

# タイミングを表示してビデオを再生
step_time = 1e6*sim_time/n_steps
step_fps = n_steps/sim_time
print(f'simulation: {step_time:5.3g} μs/step  ({step_fps:5.0f}Hz)')
frame_time = 1e6*render_time/n_frames
frame_fps = n_frames/render_time
print(f'rendering:  {frame_time:5.3g} μs/frame ({frame_fps:5.0f}Hz)')
print('\n')

# ビデオを表示
media.show_video(frames, fps=framerate)

レンダリングは物理シミュレーションよりも**はるかに**遅いことに注意してください。

## カオス
これは [カオス](https://en.wikipedia.org/wiki/Chaos_theory) システムです（初期条件のわずかな摂動が急速に蓄積されます）。

In [0]:
PERTURBATION = 1e-7
SIM_DURATION = 10 # 秒
NUM_REPEATS = 8

# 事前割り当て
n_steps = int(SIM_DURATION / model.opt.timestep)
sim_time = np.zeros(n_steps)
angle = np.zeros(n_steps)
energy = np.zeros(n_steps)

# プロット軸を準備
_, ax = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

# わずかに異なる初期条件でNUM_REPEATS回シミュレート
for _ in range(NUM_REPEATS):
  # 初期化
  mujoco.mj_resetData(model, data)
  data.qvel[0] = 10 # rootジョイントの速度
  # 初期速度を摂動
  data.qvel[:] += PERTURBATION * np.random.randn(model.nv)

  # シミュレート
  for i in range(n_steps):
    mujoco.mj_step(model, data)
    sim_time[i] = data.time
    angle[i] = data.joint('root').qpos
    energy[i] = data.energy[0] + data.energy[1]

  # プロット
  ax[0].plot(sim_time, angle)
  ax[1].plot(sim_time, energy)

# プロットを仕上げ
ax[0].set_title('root angle')
ax[0].set_ylabel('radian')
ax[1].set_title('total energy')
ax[1].set_ylabel('Joule')
ax[1].set_xlabel('second')
plt.tight_layout()

## タイムステップと精度
**質問:** なぜエネルギーが変化しているのでしょうか？摩擦や減衰がないので、このシステムはエネルギーを保存するはずです。

**答え:** 時間の離散化のためです。

タイムステップを減らすと、より良い精度とより良いエネルギー保存が得られます。

In [0]:
SIM_DURATION = 10 # (秒)
TIMESTEPS = np.power(10, np.linspace(-2, -4, 5))

# プロット軸を準備
_, ax = plt.subplots(1, 1)

for dt in TIMESTEPS:
   # タイムステップを設定、表示
  model.opt.timestep = dt

  # 割り当て
  n_steps = int(SIM_DURATION / model.opt.timestep)
  sim_time = np.zeros(n_steps)
  energy = np.zeros(n_steps)

  # 初期化
  mujoco.mj_resetData(model, data)
  data.qvel[0] = 9 # rootジョイントの速度

  # シミュレート
  print('{} steps at dt = {:2.2g}ms'.format(n_steps, 1000*dt))
  for i in range(n_steps):
    mujoco.mj_step(model, data)
    sim_time[i] = data.time
    energy[i] = data.energy[0] + data.energy[1]

  # プロット
  ax.plot(sim_time, energy, label='timestep = {:2.2g}ms'.format(1000*dt))

# プロットを仕上げ
ax.set_title('energy')
ax.set_ylabel('Joule')
ax.set_xlabel('second')
ax.legend(frameon=True);
plt.tight_layout()

## タイムステップと発散
タイムステップを増やすと、シミュレーションはすぐに発散します。

In [0]:
SIM_DURATION = 10 # (秒)
TIMESTEPS = np.power(10, np.linspace(-2, -1.5, 7))

# プロット軸を取得
ax = plt.gca()

for dt in TIMESTEPS:
  # タイムステップを設定
  model.opt.timestep = dt

  # 割り当て
  n_steps = int(SIM_DURATION / model.opt.timestep)
  sim_time = np.zeros(n_steps)
  energy = np.zeros(n_steps) * np.nan
  speed = np.zeros(n_steps) * np.nan

  # 初期化
  mujoco.mj_resetData(model, data)
  data.qvel[0] = 11 # rootジョイントの速度を設定

  # シミュレート
  print('simulating {} steps at dt = {:2.2g}ms'.format(n_steps, 1000*dt))
  for i in range(n_steps):
    mujoco.mj_step(model, data)
    if data.warning.number.any():
      warning_index = np.nonzero(data.warning.number)[0][0]
      warning = mujoco.mjtWarning(warning_index).name
      print(f'stopped due to divergence ({warning}) at timestep {i}.\n')
      break
    sim_time[i] = data.time
    energy[i] = sum(abs(data.qvel))
    speed[i] = np.linalg.norm(data.qvel)

  # プロット
  ax.plot(sim_time, energy, label='timestep = {:2.2g}ms'.format(1000*dt))
  ax.set_yscale('log')

# プロットを仕上げ
ax.set_ybound(1, 1e3)
ax.set_title('energy')
ax.set_ylabel('Joule')
ax.set_xlabel('second')
ax.legend(frameon=True, loc='lower right');
plt.tight_layout()

# 接触

ボックスと球の例に戻り、フリージョイントを追加しましょう。

In [0]:
free_body_MJCF = """
<mujoco>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
    rgb2=".2 .3 .4" width="300" height="300" mark="edge" markrgb=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="2 2" texuniform="true"
    reflectance=".2"/>
  </asset>

  <worldbody>
    <light pos="0 0 1" mode="trackcom"/>
    <geom name="ground" type="plane" pos="0 0 -.5" size="2 2 .1" material="grid" solimp=".99 .99 .01" solref=".001 1"/>
    <body name="box_and_sphere" pos="0 0 0">
      <freejoint/>
      <geom name="red_box" type="box" size=".1 .1 .1" rgba="1 0 0 1" solimp=".99 .99 .01"  solref=".001 1"/>
      <geom name="green_sphere" size=".06" pos=".1 .1 .1" rgba="0 1 0 1"/>
      <camera name="fixed" pos="0 -.6 .3" xyaxes="1 0 0 0 1 2"/>
      <camera name="track" pos="0 -.6 .3" xyaxes="1 0 0 0 1 2" mode="track"/>
    </body>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(free_body_MJCF)
data = mujoco.MjData(model)
height = 400
width = 600

with mujoco.Renderer(model, height, width) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data, "fixed")

  media.show_image(renderer.render())

このボディが床で転がるのをスローモーションでレンダリングしながら、接触点と力を可視化しましょう。

In [0]:
n_frames = 200
height = 240
width = 320
frames = []

# 接触フレームと力を可視化、ボディを透明に
options = mujoco.MjvOption()
mujoco.mjv_defaultOption(options)
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True
options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True

# 接触可視化要素のスケールを調整
model.vis.scale.contactwidth = 0.1
model.vis.scale.contactheight = 0.03
model.vis.scale.forcewidth = 0.05
model.vis.map.force = 0.3

# ランダムな初期回転速度
mujoco.mj_resetData(model, data)
data.qvel[3:6] = 5*np.random.randn(3)

# シミュレートしてビデオを表示
with mujoco.Renderer(model, height, width) as renderer:
  for i in range(n_frames):
    while data.time < i/120.0: # 1/4倍速
      mujoco.mj_step(model, data)
    renderer.update_scene(data, "track", options)
    frame = renderer.render()
    frames.append(frame)

media.show_video(frames, fps=30)

## 接触力の分析

上記のシミュレーションを再実行して（異なるランダムな初期条件で）、接触に関連するいくつかの値をプロットしましょう。

In [0]:
n_steps = 499

# 割り当て
sim_time = np.zeros(n_steps)
ncon = np.zeros(n_steps)
force = np.zeros((n_steps,3))
velocity = np.zeros((n_steps, model.nv))
penetration = np.zeros(n_steps)
acceleration = np.zeros((n_steps, model.nv))
forcetorque = np.zeros(6)

# ランダムな初期回転速度
mujoco.mj_resetData(model, data)
data.qvel[3:6] = 2*np.random.randn(3)

# シミュレートしてデータを保存
for i in range(n_steps):
  mujoco.mj_step(model, data)
  sim_time[i] = data.time
  ncon[i] = data.ncon
  velocity[i] = data.qvel[:]
  acceleration[i] = data.qacc[:]
  # アクティブな接触を反復し、力と距離を保存
  for j,c in enumerate(data.contact):
    mujoco.mj_contactForce(model, data, j, forcetorque)
    force[i] += forcetorque[0:3]
    penetration[i] = min(penetration[i], c.dist)
  # 次のようにすることもできます
  # force[i] += data.qfrc_constraint[0:3]
  # なぜか分かりますか？

# プロット
_, ax = plt.subplots(3, 2, sharex=True, figsize=(10, 10))

lines = ax[0,0].plot(sim_time, force)
ax[0,0].set_title('contact force')
ax[0,0].set_ylabel('Newton')
ax[0,0].legend(list(lines), ('normal z', 'friction x', 'friction y'));

ax[1,0].plot(sim_time, acceleration)
ax[1,0].set_title('acceleration')
ax[1,0].set_ylabel('(meter,radian)/s/s')
ax[1,0].legend(['ax', 'ay', 'az', 'αx', 'αy', 'αz'])

ax[2,0].plot(sim_time, velocity)
ax[2,0].set_title('velocity')
ax[2,0].set_ylabel('(meter,radian)/s')
ax[2,0].set_xlabel('second')
ax[2,0].legend(['vx', 'vy', 'vz', 'ωx', 'ωy', 'ωz'])

ax[0,1].plot(sim_time, ncon)
ax[0,1].set_title('number of contacts')
ax[0,1].set_yticks(range(6))

ax[1,1].plot(sim_time, force[:,0])
ax[1,1].set_yscale('log')
ax[1,1].set_title('normal (z) force - log scale')
ax[1,1].set_ylabel('Newton')
z_gravity = -model.opt.gravity[2]
mg = model.body("box_and_sphere").mass[0] * z_gravity
mg_line = ax[1,1].plot(sim_time, np.ones(n_steps)*mg, label='m*g', linewidth=1)
ax[1,1].legend()

ax[2,1].plot(sim_time, 1000*penetration)
ax[2,1].set_title('penetration depth')
ax[2,1].set_ylabel('millimeter')
ax[2,1].set_xlabel('second')

plt.tight_layout()

## 摩擦

摩擦値を変更する効果を見てみましょう。

In [0]:
MJCF = """
<mujoco>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
    <material name="grid" texture="grid" texrepeat="6 6"
     texuniform="true" reflectance=".2"/>
     <material name="wall" rgba='.5 .5 .5 1'/>
  </asset>

  <default>
    <geom type="box" size=".05 .05 .05" />
    <joint type="free"/>
  </default>

  <worldbody>
    <light name="light" pos="-.2 0 1"/>
    <geom name="ground" type="plane" size=".5 .5 10" material="grid"
     zaxis="-.3 0 1" friction=".1"/>
    <camera name="y" pos="-.1 -.6 .3" xyaxes="1 0 0 0 1 2"/>
    <body pos="0 0 .1">
      <joint/>
      <geom/>
    </body>
    <body pos="0 .2 .1">
      <joint/>
      <geom friction=".33"/>
    </body>
  </worldbody>

</mujoco>
"""
n_frames = 60
height = 300
width = 300
frames = []

# 読み込み
model = mujoco.MjModel.from_xml_string(MJCF)
data = mujoco.MjData(model)

# シミュレートしてビデオを表示
with mujoco.Renderer(model, height, width) as renderer:
  mujoco.mj_resetData(model, data)
  for i in range(n_frames):
    while data.time < i/30.0:
      mujoco.mj_step(model, data)
    renderer.update_scene(data, "y")
    frame = renderer.render()
    frames.append(frame)

media.show_video(frames, fps=30)

# 腱、アクチュエータ、センサー

In [0]:
MJCF = """
<mujoco>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
    <material name="grid" texture="grid" texrepeat="1 1"
     texuniform="true" reflectance=".2"/>
  </asset>

  <worldbody>
    <light name="light" pos="0 0 1"/>
    <geom name="floor" type="plane" pos="0 0 -.5" size="2 2 .1" material="grid"/>
    <site name="anchor" pos="0 0 .3" size=".01"/>
    <camera name="fixed" pos="0 -1.3 .5" xyaxes="1 0 0 0 1 2"/>

    <geom name="pole" type="cylinder" fromto=".3 0 -.5 .3 0 -.1" size=".04"/>
    <body name="bat" pos=".3 0 -.1">
      <joint name="swing" type="hinge" damping="1" axis="0 0 1"/>
      <geom name="bat" type="capsule" fromto="0 0 .04 0 -.3 .04"
       size=".04" rgba="0 0 1 1"/>
    </body>

    <body name="box_and_sphere" pos="0 0 0">
      <joint name="free" type="free"/>
      <geom name="red_box" type="box" size=".1 .1 .1" rgba="1 0 0 1"/>
      <geom name="green_sphere"  size=".06" pos=".1 .1 .1" rgba="0 1 0 1"/>
      <site name="hook" pos="-.1 -.1 -.1" size=".01"/>
      <site name="IMU"/>
    </body>
  </worldbody>

  <tendon>
    <spatial name="wire" limited="true" range="0 0.35" width="0.003">
      <site site="anchor"/>
      <site site="hook"/>
    </spatial>
  </tendon>

  <actuator>
    <motor name="my_motor" joint="swing" gear="1"/>
  </actuator>

  <sensor>
    <accelerometer name="accelerometer" site="IMU"/>
  </sensor>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(MJCF)
data = mujoco.MjData(model)
height = 480
width = 480

with mujoco.Renderer(model, height, width) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data, "fixed")

  media.show_image(renderer.render())

アクチュエータ付きバットと受動的な「ピニャータ」：

In [0]:
n_frames = 180
height = 240
width = 320
frames = []
fps = 60.0
times = []
sensordata = []

# 一定のアクチュエータ信号
mujoco.mj_resetData(model, data)
data.ctrl = 20

# シミュレートしてビデオを表示
with mujoco.Renderer(model, height, width) as renderer:
  for i in range(n_frames):
    while data.time < i/fps:
      mujoco.mj_step(model, data)
      times.append(data.time)
      sensordata.append(data.sensor('accelerometer').data.copy())
    renderer.update_scene(data, "fixed")
    frame = renderer.render()
    frames.append(frame)

media.show_video(frames, fps=fps)

加速度計センサーによって測定された値をプロットしましょう。

In [0]:
ax = plt.gca()

ax.plot(np.asarray(times), np.asarray(sensordata), label=[f"axis {v}" for v in ['x', 'y', 'z']])

# プロットを仕上げ
ax.set_title('Accelerometer values')
ax.set_ylabel('meter/second^2')
ax.set_xlabel('second')
ax.legend(frameon=True, loc='lower right')
plt.tight_layout()

ボディがバットに当たられた瞬間が、加速度計の測定値に明確に表れていることに注意してください。

# 高度なレンダリング

ジョイントの可視化と同様に、追加のレンダリングオプションは `render` メソッドのパラメータとして公開されています。

最初のモデルを戻しましょう。

In [0]:
xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <body name="box_and_sphere" euler="0 0 -30">
      <joint name="swing" type="hinge" axis="1 -1 0" pos="-.2 -.2 -.2"/>
      <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
      <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
    </body>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

with mujoco.Renderer(model) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data)
  media.show_image(renderer.render())

In [0]:
#@title 透過とフレーム可視化を有効化 {vertical-output: true}

scene_option.frame = mujoco.mjtFrame.mjFRAME_GEOM
scene_option.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True
with mujoco.Renderer(model) as renderer:
  renderer.update_scene(data, scene_option=scene_option)
  frame = renderer.render()
  media.show_image(frame)

In [0]:
#@title 深度レンダリング {vertical-output: true}

with mujoco.Renderer(model) as renderer:
  # 深度レンダリングを有効化
  renderer.enable_depth_rendering()

  # シーンをリセット
  renderer.update_scene(data)

  # depthは浮動小数点配列で、単位はメートル
  depth = renderer.render()

  # 最も近い値を原点にシフト
  depth -= depth.min()
  # 近い光線の平均距離の2倍でスケール
  depth /= 2*depth[depth <= 1].mean()
  # [0, 255]にスケール
  pixels = 255*np.clip(depth, 0, 1)

  media.show_image(pixels.astype(np.uint8))

In [0]:
#@title セグメンテーションレンダリング {vertical-output: true}

with mujoco.Renderer(model) as renderer:
  renderer.disable_depth_rendering()

  # セグメンテーションレンダリングを有効化
  renderer.enable_segmentation_rendering()

  # シーンをリセット
  renderer.update_scene(data)

  seg = renderer.render()

  # 最初のチャネルの内容を表示。これにはオブジェクトIDが含まれます
  # 2番目のチャネル seg[:, :, 1] にはオブジェクトタイプが含まれます
  geom_ids = seg[:, :, 0]
  # 無限大は-1にマップされます
  geom_ids = geom_ids.astype(np.float64) + 1
  # [0, 1]にスケール
  geom_ids = geom_ids / geom_ids.max()
  pixels = 255*geom_ids
  media.show_image(pixels.astype(np.uint8))

## カメラ行列

カメラ行列の説明については、Wikipediaの [Camera matrix](https://en.wikipedia.org/wiki/Camera_matrix) の記事を参照してください。

In [0]:
def compute_camera_matrix(renderer, data):
  """3x4カメラ行列を返します。"""
  # カメラが'free'カメラの場合、位置と向きを
  # シーンデータ構造から取得します。ステレオカメラなので
  # 左右のチャネルで平均を取ります。注: `scene.camera`の内容が
  # 正しいことを保証するために`self.update()`を呼び出します。
  renderer.update_scene(data)
  pos = np.mean([camera.pos for camera in renderer.scene.camera], axis=0)
  z = -np.mean([camera.forward for camera in renderer.scene.camera], axis=0)
  y = np.mean([camera.up for camera in renderer.scene.camera], axis=0)
  rot = np.vstack((np.cross(y, z), y, z))
  fov = model.vis.global_.fovy

  # 変換行列 (4x4)
  translation = np.eye(4)
  translation[0:3, 3] = -pos

  # 回転行列 (4x4)
  rotation = np.eye(4)
  rotation[0:3, 0:3] = rot

  # 焦点変換行列 (3x4)
  focal_scaling = (1./np.tan(np.deg2rad(fov)/2)) * renderer.height / 2.0
  focal = np.diag([-focal_scaling, focal_scaling, 1.0, 0])[0:3, :]

  # 画像行列 (3x3)
  image = np.eye(3)
  image[0, 2] = (renderer.width - 1) / 2.0
  image[1, 2] = (renderer.height - 1) / 2.0
  return image @ focal @ rotation @ translation

In [0]:
#@title ワールド座標からカメラ座標への投影 {vertical-output: true}

with mujoco.Renderer(model) as renderer:
  renderer.disable_segmentation_rendering()
  # シーンをリセット
  renderer.update_scene(data)

  # ボックスの角のワールド座標を取得
  box_pos = data.geom_xpos[model.geom('red_box').id]
  box_mat = data.geom_xmat[model.geom('red_box').id].reshape(3, 3)
  box_size = model.geom_size[model.geom('red_box').id]
  offsets = np.array([-1, 1]) * box_size[:, None]
  xyz_local = np.stack(list(itertools.product(*offsets))).T
  xyz_global = box_pos[:, None] + box_mat @ xyz_local

  # カメラ行列は同次座標 [x, y, z, 1] ベクトルを乗算します
  corners_homogeneous = np.ones((4, xyz_global.shape[1]), dtype=float)
  corners_homogeneous[:3, :] = xyz_global

  # カメラ行列を取得
  m = compute_camera_matrix(renderer, data)

  # ワールド座標をピクセル空間に投影します。参照:
  # https://en.wikipedia.org/wiki/3D_projection#Mathematical_formula
  xs, ys, s = m @ corners_homogeneous
  # xとyはピクセル座標系にあります
  x = xs / s
  y = ys / s

  # カメラビューをレンダリングして投影された角座標をオーバーレイします
  pixels = renderer.render()
  fig, ax = plt.subplots(1, 1)
  ax.imshow(pixels)
  ax.plot(x, y, '+', c='w')
  ax.set_axis_off()

## シーンの変更

`mjvScene` に任意のジオメトリを追加しましょう。

In [0]:
def get_geom_speed(model, data, geom_name):
  """geomの速度を返します。"""
  geom_vel = np.zeros(6)
  geom_type = mujoco.mjtObj.mjOBJ_GEOM
  geom_id = data.geom(geom_name).id
  mujoco.mj_objectVelocity(model, data, geom_type, geom_id, geom_vel, 0)
  return np.linalg.norm(geom_vel)

def add_visual_capsule(scene, point1, point2, radius, rgba):
  """mjvSceneに1つのカプセルを追加します。"""
  if scene.ngeom >= scene.maxgeom:
    return
  scene.ngeom += 1  # ngeomをインクリメント
  # 新しいカプセルを初期化し、mjv_connectorを使ってシーンに追加
  mujoco.mjv_initGeom(scene.geoms[scene.ngeom-1],
                      mujoco.mjtGeom.mjGEOM_CAPSULE, np.zeros(3),
                      np.zeros(3), np.zeros(9), rgba.astype(np.float32))
  mujoco.mjv_connector(scene.geoms[scene.ngeom-1],
                       mujoco.mjtGeom.mjGEOM_CAPSULE, radius,
                       point1, point2)

 # 時間、位置、速度のトレース
times = []
positions = []
speeds = []
offset = model.jnt_axis[0]/16  # ジョイント軸に沿ったオフセット

def modify_scene(scn):
  """位置トレースを描画、速度が幅と色を変更します。"""
  if len(positions) > 1:
    for i in range(len(positions)-1):
      rgba=np.array((np.clip(speeds[i]/10, 0, 1),
                     np.clip(1-speeds[i]/10, 0, 1),
                     .5, 1.))
      radius=.003*(1+speeds[i])
      point1 = positions[i] + offset*times[i]
      point2 = positions[i+1] + offset*times[i+1]
      add_visual_capsule(scn, point1, point2, radius, rgba)

duration = 6    # (秒)
framerate = 30  # (Hz)

# シミュレートしてビデオを表示
frames = []

# 状態と時間をリセット
mujoco.mj_resetData(model, data)
mujoco.mj_forward(model, data)

with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    # トレースにデータを追加
    positions.append(data.geom_xpos[data.geom("green_sphere").id].copy())
    times.append(data.time)
    speeds.append(get_geom_speed(model, data, "green_sphere"))
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data)
      modify_scene(renderer.scene)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

## 同じシーン内の複数のフレーム

同じジオメトリを複数回描画したい場合があります。たとえば、モデルがモーションキャプチャからの状態を追跡している場合、モデルの横にデータを視覚化すると便利です。呼び出しごとにシーンをクリアする `mjv_updateScene`（`Renderer` の `update_scene` メソッドによって呼び出される）とは異なり、`mjv_addGeoms` は既存のシーンに視覚geomを追加します。

In [0]:
# MuJoCoの標準ヒューマノイドモデルを取得
print('Getting MuJoCo humanoid XML description from GitHub:')
!git clone https://github.com/google-deepmind/mujoco
with open('mujoco/model/humanoid/humanoid.xml', 'r') as f:
  xml = f.read()

# モデルを読み込み、2つのMjDataを作成
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
data2 = mujoco.MjData(model)

# エピソードパラメータ
duration = 3       # (秒)
framerate = 60     # (Hz)
data.qpos[0:2] = [-.5, -.5]  # 初期x-y位置 (m)
data.qvel[2] = 4   # 初期垂直速度 (m/s)
ctrl_phase = 2 * np.pi * np.random.rand(model.nu)  # 制御位相
ctrl_freq = 1     # 制御周波数

# 「ゴースト」モデルの視覚オプション
vopt2 = mujoco.MjvOption()
vopt2.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True  # 透明
pert = mujoco.MjvPerturb()  # 空のMjvPerturbオブジェクト
# 動的オブジェクト（ヒューマノイド）のみが必要です。静的オブジェクト（床）は
# 再描画すべきではありません。mjtCatBitフラグでこれを実現できますが、同等に
# mjtVisFlag.mjVIS_STATICを使用することもできます
catmask = mujoco.mjtCatBit.mjCAT_DYNAMIC

# シミュレートしてレンダリング
frames = []
with mujoco.Renderer(model, 480, 640) as renderer:
  while data.time < duration:
    # 正弦波制御信号
    data.ctrl = np.sin(ctrl_phase + 2 * np.pi * data.time * ctrl_freq)
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      # これは`data`から通常のヒューマノイドを描画します
      renderer.update_scene(data)

      # qposをdata2にコピーし、ヒューマノイドを横に移動し、mj_forwardを呼び出します
      data2.qpos = data.qpos
      data2.qpos[0] += 1.5
      data2.qpos[1] += 1
      mujoco.mj_forward(model, data2)

      # mjv_addGeomsを呼び出してゴーストヒューマノイドをシーンに追加します
      mujoco.mjv_addGeoms(model, data2, vopt2, pert, catmask, renderer.scene)

      # レンダリングしてフレームを追加
      pixels = renderer.render()
      frames.append(pixels)

# ビデオを半分の実時間でレンダリング
media.show_video(frames, fps=framerate/2)

## カメラ制御

カメラは動的に制御して、映画的な効果を実現できます。以下の3つのセルを実行して、静的カメラと移動カメラからのレンダリングの違いを確認してください。

カメラ制御コードは、固定点を周回する軌道と、移動するオブジェクトを追跡する軌道の2つの間をスムーズに移行します。コード内のパラメータ値は、低解像度ビデオで素早く反復することによって取得されました。

In [0]:
#@title Load the "dominos" model

dominos_xml = """
<mujoco>
  <asset>
    <texture type="skybox" builtin="gradient" rgb1=".3 .5 .7" rgb2="0 0 0" width="32" height="512"/>
    <texture name="grid" type="2d" builtin="checker" width="512" height="512" rgb1=".1 .2 .3" rgb2=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="2 2" texuniform="true" reflectance=".2"/>
  </asset>

  <statistic meansize=".01"/>

  <visual>
    <global offheight="2160" offwidth="3840"/>
    <quality offsamples="8"/>
  </visual>

  <default>
    <geom type="box" solref=".005 1"/>
    <default class="static">
      <geom rgba=".3 .5 .7 1"/>
    </default>
  </default>

  <option timestep="5e-4"/>

  <worldbody>
    <light pos=".3 -.3 .8" mode="trackcom" diffuse="1 1 1" specular=".3 .3 .3"/>
    <light pos="0 -.3 .4" mode="targetbodycom" target="box" diffuse=".8 .8 .8" specular=".3 .3 .3"/>
    <geom name="floor" type="plane" size="3 3 .01" pos="-0.025 -0.295  0" material="grid"/>
    <geom name="ramp" pos=".25 -.45 -.03" size=".04 .1 .07" euler="-30 0 0" class="static"/>
    <camera name="top" pos="-0.37 -0.78 0.49" xyaxes="0.78 -0.63 0 0.27 0.33 0.9"/>

    <body name="ball" pos=".25 -.45 .1">
      <freejoint name="ball"/>
      <geom name="ball" type="sphere" size=".02" rgba=".65 .81 .55 1"/>
    </body>

    <body pos=".26 -.3 .03" euler="0 0 -90.0">
      <freejoint/>
      <geom size=".0015 .015 .03" rgba="1 .5 .5 1"/>
    </body>

    <body pos=".26 -.27 .04" euler="0 0 -81.0">
      <freejoint/>
      <geom size=".002 .02 .04" rgba="1 1 .5 1"/>
    </body>

    <body pos=".24 -.21 .06" euler="0 0 -63.0">
      <freejoint/>
      <geom size=".003 .03 .06" rgba=".5 1 .5 1"/>
    </body>

    <body pos=".2 -.16 .08" euler="0 0 -45.0">
      <freejoint/>
      <geom size=".004 .04 .08" rgba=".5 1 1 1"/>
    </body>

    <body pos=".15 -.12 .1" euler="0 0 -27.0">
      <freejoint/>
      <geom size=".005 .05 .1" rgba=".5 .5 1 1"/>
    </body>

    <body pos=".09 -.1 .12" euler="0 0 -9.0">
      <freejoint/>
      <geom size=".006 .06 .12" rgba="1 .5 1 1"/>
    </body>

    <body name="seasaw_wrapper" pos="-.23 -.1 0" euler="0 0 30">
      <geom size=".01 .01 .015" pos="0 .05 .015" class="static"/>
      <geom size=".01 .01 .015" pos="0 -.05 .015" class="static"/>
      <geom type="cylinder" size=".01 .0175" pos="-.09 0 .0175" class="static"/>
      <body name="seasaw" pos="0 0 .03">
        <joint axis="0 1 0"/>
        <geom type="cylinder" size=".005 .039" zaxis="0 1 0" rgba=".84 .15 .33 1"/>
        <geom size=".1 .02 .005" pos="0 0 .01" rgba=".84 .15 .33 1"/>
      </body>
    </body>

    <body name="box" pos="-.3 -.14 .05501" euler="0 0 -30">
      <freejoint name="box"/>
      <geom name="box" size=".01 .01 .01" rgba=".0 .7 .79 1"/>
    </body>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(dominos_xml)
data = mujoco.MjData(model)


In [0]:
#@title 固定カメラからレンダリング

duration = 2.5  # (秒)
framerate = 60  # (Hz)
height = 1024
width = 1440

# シミュレートしてビデオを表示
frames = []
mujoco.mj_resetData(model, data)  # 状態と時間をリセット
with mujoco.Renderer(model, height, width) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, camera='top')
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

In [0]:
#@title 移動カメラからレンダリング

duration = 3  # (秒)
height = 1024
width = 1440

# ボックスが投げられる時間を見つける（速度 > 2cm/s）
throw_time = 0.0
mujoco.mj_resetData(model, data)
while data.time < duration and not throw_time:
  mujoco.mj_step(model, data)
  box_speed = np.linalg.norm(data.joint('box').qvel[:3])
  if box_speed > 0.02:
    throw_time = data.time
assert throw_time > 0

def mix(time, t0=0.0, width=1.0):
  """シグモイド混合関数。"""
  t = (time - t0) / width
  s = 1 / (1 + np.exp(-t))
  return 1 - s, s

def unit_cos(t):
  """(0,0)から(1,1)への単位コサインシグモイド。"""
  return 0.5 - np.cos(np.pi*np.clip(t, 0, 1))/2

def orbit_motion(t):
  """軌道軌跡を返します。"""
  distance = 0.9
  azimuth = 140 + 100 * unit_cos(t)
  elevation = -30
  lookat = data.geom('floor').xpos.copy()
  return distance, azimuth, elevation, lookat

def track_motion():
  """ボックス追跡軌跡を返します。"""
  distance = 0.08
  azimuth = 280
  elevation = -10
  lookat = data.geom('box').xpos.copy()
  return distance, azimuth, elevation, lookat

def cam_motion():
  """シグモイド混合された{軌道、ボックス追跡}軌跡を返します。"""
  d0, a0, e0, l0 = orbit_motion(data.time / throw_time)
  d1, a1, e1, l1 = track_motion()
  mix_time = 0.3
  w0, w1 = mix(data.time, throw_time, mix_time)
  return w0*d0+w1*d1, w0*a0+w1*a1, w0*e0+w1*e1, w0*l0+w1*l1

# カメラを作成
cam = mujoco.MjvCamera()
mujoco.mjv_defaultCamera(cam)

# シミュレートしてビデオを表示
framerate = 60  # (Hz)
slowdown = 4    # 4倍スロー
mujoco.mj_resetData(model, data)
frames = []
with mujoco.Renderer(model, height, width) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate * slowdown:
      cam.distance, cam.azimuth, cam.elevation, cam.lookat = cam_motion()
      renderer.update_scene(data, cam)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)